# MD Simulations

This example shows how ensemble averages can be computed from MD simulations, such that their gradient with respect to force field parameters can be computed through backpropagation.

We start by parameterizing the set of molecules that will appear in our simulation boxes: 

In [1]:
import openff.interchange
import openff.toolkit

import smee

interchanges = [
    openff.interchange.Interchange.from_smirnoff(
        openff.toolkit.ForceField("openff-2.0.0.offxml"),
        openff.toolkit.Molecule.from_smiles(smiles).to_topology(),
    )
    for smiles in ("O", "CO")
]

tensor_ff, topologies = smee.convert_interchange(interchanges)

We will also flag that the vdW parameter gradients are required:

In [2]:
vdw_potential = tensor_ff.potentials_by_type["vdW"]
vdw_potential.parameters.requires_grad = True

We then define the full simulation boxes that we wish to simulate:

In [3]:
# define a periodic box containing 216 water molecules
system_water = smee.ff.TensorSystem([topologies[0]], [216], is_periodic=True)
# define a periodic box containing 128 methanol molecules
system_methanol = smee.ff.TensorSystem([topologies[1]], [216], True)
# define a periodic box containing 128 water molecules and 128 methanol molecules
system_mixture = smee.ff.TensorSystem(topologies, [128, 128], True)

A tensor system is simply a wrapper around a set of topology objects that define parameters applied to individual molecules, and the number of copies of that topology that should be present similar to GROMACS topologies. The `is_periodic` flag indicates whether the system should be simulated in a periodic box.

We then also must define the simulation protocol that will be used to run the simulations. This consists of a config object that defines how to generate the system coordinates using PACKMOL, the set of energy minimisations /simulations to run as equilibration, and finally the configuration of the production simulation:

In [4]:
import openmm.unit

import smee.mm


def simulation_protocol():
    temperature = 298.15 * openmm.unit.kelvin
    pressure = 1.0 * openmm.unit.atmosphere

    coords_config = smee.mm.GenerateCoordsConfig()
    # we can run an arbitrary number of equilibration simulations / minimizations.
    # all generated data will be discarded, but the final coordinates will be used
    # to initialize the production simulation
    equilibrate_config = [
        smee.mm.MinimizationConfig(),
        # short NVT equilibration simulation
        smee.mm.SimulationConfig(
            temperature=temperature,
            pressure=None,
            n_steps=1,
            timestep=1.0 * openmm.unit.femtosecond,
        ),
        # short NPT equilibration simulation
        smee.mm.SimulationConfig(
            temperature=temperature,
            pressure=pressure,
            n_steps=1,
            timestep=1.0 * openmm.unit.femtosecond,
        ),
    ]
    # long NPT production simulation
    production_config = smee.mm.SimulationConfig(
        temperature=temperature,
        pressure=pressure,
        n_steps=2,
        timestep=2.0 * openmm.unit.femtosecond,
    )
    # store coords and values every 2 ps
    report_interval = 1

    return coords_config, equilibrate_config, production_config, report_interval

Computing the ensemble averages is then as simple as:

In [5]:
# run simulations of each system and compute ensemble averages over the trajectories
# of the potential energy, volume, and density
water_avgs = smee.mm.compute_ensemble_averages(
    system_water, tensor_ff, *simulation_protocol()
)
methanol_avgs = smee.mm.compute_ensemble_averages(
    system_methanol, tensor_ff, *simulation_protocol()
)
mixture_avgs = smee.mm.compute_ensemble_averages(
    system_mixture, tensor_ff, *simulation_protocol()
)

FORWARD
FORWARD
FORWARD


Each of the returned values is a dictionary of ensemble averages computed over the simulated production trajectory. This currently includes the potential energy, volume, and density of the system.

These averages can be used in a loss function

In [6]:
# define some MOCK data and loss function
mock_water_density = 1.0  # g/mL
mock_methanol_density = 0.791  # g/mL

mock_enthalpy_of_mixing = 0.891  # g/mL

loss = (water_avgs["density"] - mock_water_density) ** 2
loss += (methanol_avgs["density"] - mock_methanol_density) ** 2

mixture_enthalpy = mixture_avgs["enthalpy"] / 256

water_enthalpy = water_avgs["enthalpy"] / 128
methanol_enthalpy = methanol_avgs["enthalpy"] / 128

enthalpy_of_mixing = mixture_enthalpy - (0.5 * water_enthalpy + 0.5 * methanol_enthalpy)
loss += (enthalpy_of_mixing - mock_enthalpy_of_mixing) ** 2

and the gradient of this loss function with respect to the force field parameters can be computed through backpropagation:

In [7]:
loss.backward()

epsilon_col = vdw_potential.parameter_cols.index("epsilon")
sigma_col = vdw_potential.parameter_cols.index("sigma")

print("VdW Ɛ Gradients", vdw_potential.parameters.grad[:, epsilon_col])
print("VdW σ Gradients", vdw_potential.parameters.grad[:, sigma_col])

BACKWARD
BACKWARD
BACKWARD
VdW Ɛ Gradients tensor([ 14.6547, -56.3339,  -5.4367,  -1.8464,  74.0201, -53.0496],
       dtype=torch.float64)
VdW σ Gradients tensor([ 1.5595e+01,  0.0000e+00,  1.0379e+01,  1.3719e+01,  1.6765e+01,
        -1.2333e-02], dtype=torch.float64)
